In [18]:
#1. 리뷰 작성자
#2. 리뷰 내용
#3. 리뷰 작성 일자
#4. 공감횟수
#5. 비공감횟수 추출

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import numpy
import pandas as pd
import random
import os

print('<<네이버 뉴스 기사 댓글 정보 수집하기>>')
print('\n')

query_txt = '뉴스기사댓글'
query_url = input('1. 크롤링할 뉴스의 url을 입력하세요: ')
cnt = int(input('2. 크롤링 할 건수는 몇건인가요?(10단위): '))
page_cnt = int(cnt/20)+1

file_dir = 'C:/Users/YEO/Desktop/크롤링 결과/' #파일 저장을 원하는 폴더의 dir

#파일 위치와 이름 설정
p_time = time.localtime()
s = "%04d-%02d-%02d-%02d시-%02d분-%02d초" % (p_time.tm_year, p_time.tm_mon, p_time.tm_mday, p_time.tm_hour, p_time.tm_min, p_time.tm_sec)

os.makedirs(file_dir + s + '-' + query_txt)

fc_name = file_dir+s+'-'+query_txt+'/'+s+'-'+query_txt+'.csv'
fx_name = file_dir+s+'-'+query_txt+'/'+s+'-'+query_txt+'.xls'

#크롬 드라이버
start_time = time.time()
chrome_path = "C:/Users/YEO/Desktop/크롤링 연습/chromedriver_91/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
driver.get(query_url)
time.sleep(2)

#댓글 갯수 찾아서 사용자 요청과 비교
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
result = soup.find('div', class_ = 'u_cbox_head').find('span', 'u_cbox_count')
result_num = result.get_text() #str타입의 전체 댓글 숫자

if cnt>int(result_num):
    cnt = int(result_num)
    
print('전체 검색 결과 건수: ', result_num, '건')
print('실제 최종 출력 건수', cnt, '건')
print('실제 출력될 최종 페이지수', page_cnt)

driver.find_element_by_xpath('''//*[@id='cbox_module']/div/div[9]/a/span[1]''').click()
time.sleep(2)

#리뷰에 대한 정보 수집 및 삭제된 것들은 예외처리
writer_id_list = [] #댓글 작성자
review_list = [] #댓글 내용
date_list = [] #댓글 날짜
gonggam_g_list = [] #긍정적 공감
gonggam_b_list = [] #부정적 공감

if cnt<=20:
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    count = 0    
    #전체 댓글 영역에서 하나씩 다 찾기
    re_all = soup.find('div', class_ = 'u_cbox_content_wrap').find('ul')
    re = re_all.find_all('li')
    
    for li in re:
        count+=1
        print('\n')
        print('%s번째 댓글 수집 중...' %count)
        
        writer_id = li.find('span', class_ = 'u_cbox_nick').get_text()
        writer_id_list.append(writer_id)
        
        #댓글 내용 수집할 때는 예외처리 해주기
        try :
            review = li.find('span', class_ = 'u_cbox_contents').get_text()
        except AttributeError :
            review = '작성자에 의해 삭제된 댓글입니다'
        else :
            review_list.append(review)
        
        write_date = li.find('span', class_ = 'u_cbox_date').get_text()
        date_list.append(write_date)
        
        gonggam = li.find('div', class_='u_cbox_recomm_set').find_all('em')
        gonggam_g = gonggam[0].text
        gonggam_g_list.append(gonggam_g)
        
        gonggam = li.find('div', class_='u_cbox_recomm_set').find_all('em')
        gonggam_b = gonggam[1].text
        gonggam_b_list.append(gonggam_b)
        
        time.sleep(0.3)
        
        if count == cnt:
            break
            
    print("%s건 완료!!" %count)
    time.sleep(3)
    
else:
    i = 1
    while (i< page_cnt):
        driver.find_element_by_xpath('''//*[@id='cbox_module']/div/div[9]/a''').click()
        time.sleep(2)
        i += 1
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    re_all = soup.find('div', class_ = 'u_cbox_content_wrap').find('ul')
    re = re_all.find_all('li')
    
    count = 0
    
    for li in re:
        count+=1
        print('\n')
        print('%s번째 댓글 수집 중...' %count)
        
        writer_id = li.find('span', class_ = 'u_cbox_nick').get_text()
        writer_id_list.append(writer_id)
        
        #댓글 내용 수집할 때는 예외처리 해주기
        try :
            review = li.find('span', class_ = 'u_cbox_contents').get_text()
        except AttributeError :
            review = '작성자에 의해 삭제된 댓글입니다'
        else :
            review_list.append(review)
        
        write_date = li.find('span', class_ = 'u_cbox_date').get_text()
        date_list.append(write_date)
        
        gonggam = li.find('div', class_='u_cbox_recomm_set').find_all('em')
        gonggam_g = gonggam[0].text
        gonggam_g_list.append(gonggam_g)
        
        gonggam = li.find('div', class_='u_cbox_recomm_set').find_all('em')
        gonggam_b = gonggam[1].text
        gonggam_b_list.append(gonggam_b)
        
        
        time.sleep(0.3)
        
        if count == cnt:
            break
          
    print("%s건 완료!!" %count)
    time.sleep(3)
    
#csv와 xls형식으로 저장
repl_data = pd.DataFrame()
repl_data['작성자 ID'] = pd.Series(writer_id_list)
repl_data['댓글 내용'] = pd.Series(review_list)
repl_data['작성 일자'] = pd.Series(date_list)
repl_data['긍정적 공감'] = pd.Series(gonggam_g_list)
repl_data['부정적 공감'] = pd.Series(gonggam_b_list)

#csv
repl_data.to_csv(fc_name,encoding='utf-8-sig', index=False)

#excel
repl_data.to_excel(fx_name, index=False)

#최종 크롤링 관련 정보 출력
end_time = time.time()
total_time = end_time - start_time

print('\n')

print('=' *50)
print('요청된 총 %s건 중 실제 크롤링 된 것은 %s건 입니다.' %(cnt, count))
print('총 소요시간은 %s초 입니다.' %total_time)
print('=' *50)

driver.close()
    



<<네이버 뉴스 기사 댓글 정보 수집하기>>


1. 크롤링할 뉴스의 url을 입력하세요: https://news.naver.com/main/read.naver?m_view=1&includeAllCount=true&mode=LSD&mid=shm&sid1=102&oid=001&aid=0012534087
2. 크롤링 할 건수는 몇건인가요?(10단위): 30
전체 검색 결과 건수:  70 건
실제 최종 출력 건수 30 건
실제 출력될 최종 페이지수 2


1번째 댓글 수집 중...


2번째 댓글 수집 중...


3번째 댓글 수집 중...


4번째 댓글 수집 중...


5번째 댓글 수집 중...


6번째 댓글 수집 중...


7번째 댓글 수집 중...


8번째 댓글 수집 중...


9번째 댓글 수집 중...


10번째 댓글 수집 중...


11번째 댓글 수집 중...


12번째 댓글 수집 중...


13번째 댓글 수집 중...


14번째 댓글 수집 중...


15번째 댓글 수집 중...


16번째 댓글 수집 중...


17번째 댓글 수집 중...


18번째 댓글 수집 중...


19번째 댓글 수집 중...


20번째 댓글 수집 중...


21번째 댓글 수집 중...


22번째 댓글 수집 중...


23번째 댓글 수집 중...


24번째 댓글 수집 중...


25번째 댓글 수집 중...


26번째 댓글 수집 중...


27번째 댓글 수집 중...


28번째 댓글 수집 중...


29번째 댓글 수집 중...


30번째 댓글 수집 중...
30건 완료!!


c:\program files\python37\lib\site-packages\ipykernel_launcher.py:176: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.




요청된 총 30건 중 실제 크롤링 된 것은 30건 입니다.
총 소요시간은 24.766075372695923초 입니다.
